# Analyse de document Microsoft Office

Plusieurs types de fichiers sont disponibles au sein de la suite Microsoft Office :
- les **documents OLE**, que nous connaissons tous : 
  - **Excel** : ".xls"
  - **Powerpoint** : ".ppt"
  - **Word** : ".doc"
- les **documents Office Open XML "OOXML"** correspondent à des archives de documents XML (documents compressés). Ce type de document, standardisé via l'ISO/IEC 29500, a été introduit dans la suite Microsoft Office pour la première fois dans la version 2010. 
  - **Excel** : ".xlsx"
  - **Powerpoint** : ".pptx"
  - **Word** : ".docx"
- les documents RTF, ayant pour extension ".rtf"

In [ ]:
from colorama import init, Fore, Back, Style
from defang import defang
import msticpy as mp
import os
import pandas as pd
import msticpy.sectools as sectools

#Expand the width of the cells
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

mp.init_notebook(globals(), verbosity=0)
ti = mp.TILookup()
ioc_extractor = sectools.IoCExtract()

# Chemin du fichier à analyser
officeFile = {}
officeFile['path'] = "/home/secubian/Documents/Cases/payloads_and_dumps/cancel_sub_VCP1234567890123.xlsb"  #https://tria.ge/210610-xr9avxf7f6/



----
Il est interessant dans un premier temps d'analyser les métadonnées du fichier.

In [ ]:
os.system(f"exiftool {officeFile['path']}")

## Analyse des méta données

Il est important d'extraire des informations permettant d'obtenir du contexte sur le document, telles que l'auteur, le logiciel utilisé, les signatures numériques (hash). \
La commande utilisée est : ```oleid {filepath}```.

In [ ]:
import oletools.oleid
oid = oletools.oleid.OleID(officeFile['path'])
officeFile['oleid'] = {}
indicators = oid.check()
for i in indicators:
    officeFile['oleid'][i.name] = i.value

for indicator in officeFile['oleid']:
    try:
        displayValue = False
        if (isinstance(officeFile['oleid'][indicator],bool) and officeFile['oleid'][indicator]) or (isinstance(officeFile['oleid'][indicator],int) and officeFile['oleid'][indicator]>0) or (officeFile['oleid'][indicator][0:3] == "Yes"):
            print(Fore.RED + f"[!] {indicator}: {officeFile['oleid'][indicator]}")
            displayValue = True
    except Exception as err:
        pass
    finally:
        if not displayValue: print(Fore.GREEN + f"[✓] {indicator}: {officeFile['oleid'][indicator]}")

Extraction des signatures numériques (HASH), permettant la recherche dans les bases de Threat Intelligence. \
La commande utilisée sous linux : 
```
md5sum {filepath}
sha256sum {filepath}
```

In [ ]:
import hashlib
officeFile['md5'] = hashlib.md5(officeFile['path'].encode('UTF-8')).hexdigest()
officeFile['sha256'] = hashlib.sha256(officeFile['path'].encode('UTF-8')).hexdigest()

df_ti = ti.lookup_iocs(data=[officeFile['md5'], officeFile['sha256']], providers=["VirusTotal", "OTX"])
df_ti = df_ti[df_ti['Result']==True]
df_ti = pd.json_normalize(data=df_ti[['Severity','Provider','Ioc','Details']].to_dict(orient='records'))

print(Fore.GREEN + "Microsoft Office file HASH.")
print(Fore.GREEN + f"[✓] MD5: \t{officeFile['md5'] } ")
print(Fore.GREEN + f"[✓] SHA256: \t{officeFile['sha256'] }")

if (df_ti.empty):
    print(Fore.GREEN + "[✓] No Potentially identified as malicious")
else:
    print(Fore.RED + "[!] Potentially identified as malicious")
    display(df_ti)


## Extraction et Analyse des relations externes au document analysé

Si des références externes ont été identifiées dans les métadonnées, l'usage de l'outil **oleobj** devrait permettre d'extraire les url,ip, domaines du document analysé. \
La commande utilisée est : ```oleobj -i {filepath} -l info```

In [ ]:
from oletools import oleobj

with open(officeFile['path'], 'rb') as file_handle:
    data = file_handle.read()

relationships = []
if officeFile['oleid']['External Relationships'] > 0:
    xml_parser = oleobj.XmlParser(officeFile['path'])
    for relationship, target in oleobj.find_external_relationships(xml_parser):
        did_dump = True
        print(Fore.RED + f"[!] Found relationship {relationship} with external link {defang(target)}")
        relationships.append(target)
        if target.startswith('mhtml:'):
            print("Potential exploit for CVE-2021-40444")
    for target in oleobj.find_customUI(xml_parser):
        did_dump = True
        print(Fore.RED + f"[!]Found customUI tag with external link or VBA macro {defang(target)} (possibly exploiting CVE-2021-42292)")
        relationships.append(target)

if not relationships:
    print(Fore.GREEN + "[✓] No relationships found")

\
En fonction du résultat obtenu ci-dessus, une recherche dans les bases de connaissance de menaces peut être pertinente.

In [ ]:
pd.set_option('display.max_colwidth', None)

if relationships:
    df_relationships = pd.DataFrame(relationships, columns = ['relation'])
    df_relationships = ioc_extractor.extract(data=df_relationships,columns=['relation'])
    df_ti = ti.lookup_iocs(data=df_relationships['Observable'], providers=["VirusTotal", "OTX"])
    df_ti = df_ti[df_ti['Result']==True]
    df_ti = pd.json_normalize(data=df_ti[['Severity','Provider','Ioc','Details']].to_dict(orient='records'))

    if (df_ti.empty):
        print(Fore.GREEN + "[✓] No Potentially identified as malicious")
    else:
        print(Fore.RED + "[!] Potentially identified as malicious")
        display(df_ti)
else:
    print(Fore.GREEN + "[✓] No relationships found")

## Extraction et Analyse des macros

Les macros sont des éléments permettant d'exécuter des morceaux de code à l'ouverture du document ou à lors d'événements bien précis. \
La commande utilisée est : 
- ```olevba {filepath} ```, pour obtenir une analyse complète et également le code source de la macro
- ```olevba -a {filepath} ```, pour obtenir uniquement le résultat de l'analyse
- ```olevba --reveal {filepath} ```, pour obtenir uniquement le code source de la macro


In [ ]:
# https://github.com/decalage2/oletools/wiki/olevba
from oletools.olevba import VBA_Parser, TYPE_OLE, TYPE_OpenXML, TYPE_Word2003_XML, TYPE_MHTML
vbaparser = VBA_Parser(officeFile['path'])

if vbaparser.detect_vba_macros():
    print(Fore.RED + "[!] VBA Macros found")
    results = vbaparser.analyze_macros()
    print(Fore.RED + f"[!] AutoExec keywords: {vbaparser.nb_autoexec}")
    print(Fore.RED + f"[!] IOCs: {vbaparser.nb_iocs}")
    print(Fore.RED + f"[!] Hex obfuscated strings: {vbaparser.nb_hexstrings}")
    print(Fore.RED + f"[!] Base64 obfuscated strings: {vbaparser.nb_base64strings}")
    print(Fore.RED + f"[!] Dridex obfuscated strings: {vbaparser.nb_dridexstrings}")
    print(Fore.RED + f"[!] VBA obfuscated strings: {vbaparser.nb_vbastrings}")

    print("\n")
    print(Fore.RED + f"[!] Suspicious patterns : {vbaparser.nb_suspicious}")
    for kw_type, keyword, description in results:
        print(f"[{kw_type}] - {keyword} : {description}")
else:
    print(Fore.GREEN + "[✓] No VBA Macros found")


Cette tentative se base sur les résultats obtenus précédemments. Si aucune chaine obfusquée n'a été détectée, aucun résultat ne sera obtenu.

In [ ]:
# https://github.com/decalage2/oletools/wiki/olevba
if vbaparser.nb_hexstrings or vbaparser.nb_base64strings or vbaparser.nb_dridexstrings or vbaparser.nb_vbastrings:
    print(Fore.RED + vbaparser.reveal())
else:
    print(Fore.GREEN + "[✓] No VBA obfuscated strings found")

Il est possible d'aller plus loin avec **XLMMacroDeobfuscator** en utilisant la commande : \
```xlmdeobfuscator --file {filepath}```.

In [ ]:
import os
#os.system(f"xlmdeobfuscator --file {officeFile['path']}")
os.system(f"xlmdeobfuscator -h")

## Recherche d'élément caché au sein du fichier analysé

Nous nous baserons dans un premier temps sur l'outil de DidierStevens **pecheck** pour identifier tout binaire contenu dans le fichier d'origine.

In [ ]:
import os
os.system(f"python3 /home/secubian/Documents/Tools/DFIR/DidierStevensSuite/pecheck.py -l P {officeFile['path']}")

Nous listerons dans un second temps les éléments contenus dans le document. \
Il est notamment important de chercher les sections telles que "x01oLE10nATiVe". \
source : https://clickallthethings.wordpress.com/2021/03/06/oleobject1-bin-ole10native-shellcode/

In [ ]:
import os
os.system(f"python3 /home/secubian/Documents/Tools/DFIR/DidierStevensSuite/oledump.py --storages -E '%CLSID% %CLSIDDESC%' {officeFile['path']}")

Une fois la section suspecte identifiée, il suffit de la spécifier à l'outil **oledump** afin d'extraire son contenu. \
Voici la commande : ```oledump.py -s {sectionID} {filepath}```

In [ ]:
import os
elementID = "A5"
os.system(f"python3 /home/secubian/Documents/Tools/DFIR/DidierStevensSuite/oledump.py -s {elementID} {officeFile['path']}")

Il est possible de continuer l'identification via l'outil **Binwalk** à la recherche d'éléments tels que "/embeddings/" par exemple.

In [ ]:
os.system(f"binwalk -B {officeFile['path']}")

Une fois les éventuels éléments cachés identifiés, nous utiliserons à nouveau l'outil **binwalk** ou **dd**, pour les extraire. \
Les champs seront saisis de la manière suivante : 
- if : correspond au fichier d'origine
- of : correspond au fichier de sortie
- skip : correspond à la valeur de l'élément présente dans la colonne DECIMAL 
- count : correspond à la taille de l'élément

In [ ]:
os.system(f"dd if={officeFile['path']} of=./suspicious_file bs=1 skip=6288 count=766549")

Si tel est le cas, il est possible d'aller plus loin en exportant le contenu malveillant au sein d'un fichier dédié pour continuer l'analyse.

In [ ]:
os.system(f"file {officeFile['path']}")
os.system(f"clamscan {officeFile['path']}")